In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# changed animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
#username and password and CRUD Python module name

username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Added in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    #Placed the HTML image tag in the line below into the app.layout code
    #Added in the image and centered it on the page
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    
    # Added a unique identifier with my name
    html.Center(html.H2('Breunna Bingham')),
    html.Center(html.H3('Animal Dashboard')),
    html.Hr(),
    html.Div(
        
        # Added in code for the interactive filtering options using Radio Buttons   
        dcc.RadioItems(
            id = 'rescue_type',
            options=[{'label': 'Water Rescue', 'value':'water'},
                     {'label': 'Mountain/Wilderness Rescue', 'value':'mountain'},
                     {'label': 'Disaster/Individual Rescue', 'value':'disaster'},
                     {'label': 'Reset', 'value':'reset'}
            ],
            value = 'reset',
            labelStyle={'display':'inline-block'}
        )
                 

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Set up the features for interactive data table to make it user-friendly for the client
        # copied my code from module six
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
    ),
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('rescue_type','value')])

def update_dashboard(value):
# Added code to filter the interactive data table with MongoDB queries 
  # according to the specifications from the client
         # shows all animals unfiltered
        if (value == 'reset'):
            df = pd.Dataframe.from_records(shelter.read({}))
            
          # Shows only the intact female breeds listed under water rescue within a certain age frame  
        elif (value == 'water'):
            df = pd.DataFrame.from_records(shelter.read({"breed": {"$in": ["Labrador Retriever Mix", 
                                                                   "Chesapeake Bay Retriever",
                                                                   "Newfoundland"]},
                                                    
                                                          "sex_upon_outcome": "Intact Female",
                                                          "age_upon_outcome_in_weeks":{"$gte":26}, #Greater than/ equal to 26 weeks
                                                  "$and": [{"age_upon_outcome_in_weeks":{"$lte":156}}]})) #less than/ equal to 156 weeks
            
        # Shows only the intact male breeds listed under mountain rescue within a certain age frame 
        elif (value == 'mountain'):
            df = pd.DataFrame.from_records(shelter.read({"breed": {"$in": ["German Shepherd", 
                                                                   "Alaskan Malamute",
                                                                   "Old English Sheepdog",
                                                                   "Siberian Husky",
                                                                   "Rottweiler"]},
                                                         
                                                          "sex_upon_outcome":"Intact Male",
                                                          "age_upon_outcome_in_weeks":{"$gte":26}, #Greater than/ equal to 26 weeks
                                                  "$and": [{"age_upon_outcome_in_weeks":{"$lte":156}}]})) #less than/ equal to 156 weeks
            
       # Shows only the intact male breeds listed under disaster rescue within a certain age frame                                                                                          
        elif (value =='disaster'):
            df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Doberman Pinscher", 
                                                                 "German Shepherd",
                                                                 "Golden Retriever",
                                                                 "Bloodhound",
                                                                 "Rottweiler"]},
                                                         
                                                          "sex_upon_outcome":"Intact Male",
                                                          "age_upon_outcome_in_weeks":{"$gte":20}, #Greater than/ equal to 20 weeks
                                                  "$and": [{"age_upon_outcome_in_weeks":{"$lte":300}}]})) #less than/ equal to 300 weeks
            
            
        
            
                                                                                                                                                                                      
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    
    # Added the code needed to display a pie chart of available breeds
    df = pd.DataFrame.from_dict(viewData)                                                                                            
    return [
        dcc.Graph(            
            figure = px.pie(df, names = 'breed')
        )    
    ]
                                                                                                

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])

def update_map(viewData, selected_rows):
# Added in the code for the geolocation chart
# Copied from module six assignment
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[selected_rows]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,'name'])
                ])
            ])
        ])
    ]

app